## Day 13: Mine Cart Madness

https://adventofcode.com/2018/day/13

### Part 1

Use complex numbers for coordinates as they're hashable and support addition.

In [1]:
from itertools import cycle, count
import copy


class Cart:
    def __init__(self, position, direction, next_turn):
        self.position = position
        self.direction = direction
        self.next_turn = next_turn

        
def coordinate(c):
    return(c.real, c.imag)


UP = complex(0, -1)
DOWN = complex(0, 1)
LEFT = complex(-1, 0)
RIGHT = complex(1, 0)


turn_left = dict(zip((UP, RIGHT, DOWN, LEFT), 
                     (LEFT, UP, RIGHT, DOWN)))
straight_on = dict(zip((UP, RIGHT, DOWN, LEFT),
                       (UP, RIGHT, DOWN, LEFT)))
turn_right = dict(zip((LEFT, UP, RIGHT, DOWN),
                      (UP, RIGHT, DOWN, LEFT)))

corner = {'\\': {LEFT: UP, UP: LEFT, DOWN: RIGHT, RIGHT: DOWN},
          '/': {LEFT: DOWN, DOWN: LEFT, UP: RIGHT, RIGHT: UP}}                      

cart_directions = {'<': LEFT, '>': RIGHT, '^': UP, 'v': DOWN}


def parse_mine(mine_data):
    cart_ids = count(1)
    mine = {}
    carts = {}
    for y, row in enumerate(mine_data):
        for x, c in enumerate(row.rstrip()):
            if c != ' ':
                mine[complex(x, y)] = c.translate(str.maketrans('<>^v', '--||'))
                if c in '<>^v':
                    cart_id = next(cart_ids)
                    carts[cart_id] =  Cart(complex(x, y), 
                                           cart_directions[c],
                                           cycle((turn_left, straight_on, turn_right)))
    return (mine, carts)


def next_crash(mine, carts):
    carts = copy.deepcopy(carts)
    for tick in count(1):
        # sorting coordinates puts the carts in order of movement
        for cart in sorted(carts, key=lambda c: coordinate(carts[c].position)):
            direction = carts[cart].direction
            tile = mine[carts[cart].position]
            if tile in corner:
                direction = corner[tile][direction]
            elif tile == '+':
                direction = next(carts[cart].next_turn)[direction]
            new_position = carts[cart].position + direction
            if new_position in (carts[c].position for c in carts):
                return coordinate(new_position)
            carts[cart].position = new_position
            carts[cart].direction = direction

In [2]:
test_data = r'''/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   '''.splitlines()

test_mine, test_carts = parse_mine(test_data)

In [3]:
next_crash(test_mine, test_carts)

(7.0, 3.0)

In [4]:
mine, carts = parse_mine(open('input', 'r'))

next_crash(mine, carts)

(48.0, 20.0)

Blimey, right first time. Perhaps I should write more hacky inelegant code after going to the pub.

### Part 2

In [5]:
def lone_survivor(mine, carts):
    carts = copy.deepcopy(carts)
    for tick in count(1):
        if len(carts) == 1:
            return coordinate(next(iter(carts.values())).position)
        # sorting coordinates puts the carts in order of movement
        for cart in sorted(carts, key=lambda c: coordinate(carts[c].position)):
            if cart in carts:
                direction = carts[cart].direction
                tile = mine[carts[cart].position]
                if tile in corner:
                    direction = corner[tile][direction]
                elif tile == '+':
                    direction = next(carts[cart].next_turn)[direction]
                new_position = carts[cart].position + direction
                crash = [c for c in carts if carts[c].position == new_position]
                if crash:
                    del carts[crash[0]]
                    del carts[cart]
                else:
                    carts[cart].position = new_position
                    carts[cart].direction = direction

In [6]:
test_data2 = r'''/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/'''.splitlines()

test_mine2, test_carts2 = parse_mine(test_data2)

In [7]:
lone_survivor(test_mine2, test_carts2)

(6.0, 4.0)

In [8]:
lone_survivor(mine, carts)

(59.0, 64.0)

### Post-mortem

The above is actually bugged. It's sorting by column then row when it should be sorting by row then column. Here's a correct version.

In [11]:
def next_crash(mine, carts):
    carts = copy.deepcopy(carts)
    for tick in count(1):
        # sorting coordinates puts the carts in order of movement
        for cart in sorted(carts, key=lambda c: (coordinate(carts[c].position)[1], 
                                                 coordinate(carts[c].position)[0])):
            direction = carts[cart].direction
            tile = mine[carts[cart].position]
            if tile in corner:
                direction = corner[tile][direction]
            elif tile == '+':
                direction = next(carts[cart].next_turn)[direction]
            new_position = carts[cart].position + direction
            if new_position in (carts[c].position for c in carts):
                return coordinate(new_position)
            carts[cart].position = new_position
            carts[cart].direction = direction
                    
next_crash(mine, carts)

(48.0, 20.0)

In [12]:
def lone_survivor(mine, carts):
    carts = copy.deepcopy(carts)
    for tick in count(1):
        if len(carts) == 1:
            return coordinate(next(iter(carts.values())).position)
        # sorting coordinates puts the carts in order of movement
        for cart in sorted(carts, key=lambda c: (coordinate(carts[c].position)[1], 
                                                 coordinate(carts[c].position)[0])):
            if cart in carts:
                direction = carts[cart].direction
                tile = mine[carts[cart].position]
                if tile in corner:
                    direction = corner[tile][direction]
                elif tile == '+':
                    direction = next(carts[cart].next_turn)[direction]
                new_position = carts[cart].position + direction
                crash = [c for c in carts if carts[c].position == new_position]
                if crash:
                    del carts[crash[0]]
                    del carts[cart]
                else:
                    carts[cart].position = new_position
                    carts[cart].direction = direction
                    
                    
lone_survivor(mine, carts)

(59.0, 64.0)